In [40]:
import numpy as np
import pandas as pd

In [41]:
# producing clustering dataset
from sklearn.datasets import make_blobs

n_samples = 200
n_features = 6
n_clusters = 4
# Generate random clustering dataset with 100 samples and 4 centers
X, y = make_blobs(n_samples=n_samples, n_features=n_features , centers=n_clusters, random_state=42)


df_x = pd.DataFrame(X)
df_y = pd.DataFrame(y)
df = pd.concat([df_x, df_y], axis=1)
df.columns = ['f0', 'f1','f2','f3','f4','f5','cluster']
df




,f0,f1,f2,f3,f4,f5,cluster
0,-8.762523,6.646361,2.997420,4.014394,-10.413807,9.076811,1
1,-2.994561,9.096160,6.954537,0.105904,-6.193367,-8.492825,0
2,-3.355991,7.499439,4.193364,2.829568,-6.665533,-8.125848,0
3,-0.171629,-5.403025,2.834458,-6.508950,-4.454671,-1.297056,3
4,-9.077276,6.415959,1.445529,4.916843,-9.087393,8.420642,1
...,...,...,...,...,...,...,...
195,-0.575299,-3.749960,1.270082,-7.257834,-4.160710,-3.831128,3
196,6.616100,-6.296643,-7.076346,-6.225480,-4.170132,1.999122,2
197,-1.962100,8.812093,4.422198,3.071947,-6.054211,-6.066600,0
198,-1.723541,-5.295087,0.942376,-6.049296,-4.624808,-2.326259,3


## Initialization

In [42]:
# Dictionary of history
history = {
    'U': [],
    'Z': [],
    'W': [],
    'cost': []

    }
time_step = 0

In [43]:
# initial centers randomly by choosing from hte dataset randomly
Z_initial_index = np.random.choice(range(n_samples), size=n_clusters)
Z_initial_index

array([136,  28,  16, 142])

In [44]:
# Centers of clusters of random samples of data, but they can be any random data_points not necessarily in dataset
Z_df = df.iloc[Z_initial_index,:-1]
Z_df

,f0,f1,f2,f3,f4,f5
136,-0.043706,-3.977818,4.312319,-7.899311,-2.421143,-2.474852
28,-8.425396,6.759798,1.200080,4.405139,-9.343344,8.891254
16,-3.837384,9.211147,5.378345,2.144538,-6.995275,-7.181213
142,-8.947088,7.725235,2.712444,3.760231,-9.364218,9.410789


In [45]:
# Z_df to ndarray
Z = Z_df.to_numpy()
Z

array([[-0.04370556, -3.97781759,  4.31231877, -7.89931061, -2.42114323,
        -2.47485235],
       [-8.4253963 ,  6.75979836,  1.20007984,  4.40513877, -9.34334354,
         8.89125387],
       [-3.83738367,  9.21114736,  5.37834542,  2.14453797, -6.99527547,
        -7.18121329],
       [-8.94708791,  7.72523464,  2.71244423,  3.76023108, -9.36421763,
         9.41078944]])

In [46]:
history['Z'].append(Z)

In [47]:
# Generate random weights that sum up to 1
weights = np.random.dirichlet(np.ones(n_features), size=1).squeeze()
weights


array([0.09005344, 0.00320794, 0.25596101, 0.15151899, 0.19957969,
       0.29967893])

In [48]:
weights.sum()

1.0

In [49]:
history['W'].append(weights)

In [50]:
history

{'U': [],
 'Z': [array([[-0.04370556, -3.97781759,  4.31231877, -7.89931061, -2.42114323,
          -2.47485235],
         [-8.4253963 ,  6.75979836,  1.20007984,  4.40513877, -9.34334354,
           8.89125387],
         [-3.83738367,  9.21114736,  5.37834542,  2.14453797, -6.99527547,
          -7.18121329],
         [-8.94708791,  7.72523464,  2.71244423,  3.76023108, -9.36421763,
           9.41078944]])],
 'W': [array([0.09005344, 0.00320794, 0.25596101, 0.15151899, 0.19957969,
         0.29967893])],
 'cost': []}

• U is an M × k partition matrix, ui,l is a binary variable, and ui,l = 1 indicates that record i is allocated to cluster l.  
• Z = {Z1, Z2, ..., Zk} is a set of k vectors representing the k-cluster centers.  
• W = [w1, w2, ..., wN ] is a set of weights.  
• d(xi,j, zl,j) is a distance or dissimilarity measure between object i and the center of cluster l on the jth feature.

1. P1: Fix Z and W ; solve the reduced problem for U
2. P2: Fix U and W ; solve the reduced problem for Z
3. P3: Fix U and Z ; solve the reduced problem for W

In [51]:
def weighted_distance(s1, s2, weight_vec, beta):
    """Calculate the weighted distance between two samples s1 and  s2 and based on the weights that each feature has

    Args:
        s1 (ndarray): _description_
        s2 (ndarray): _description_
        weight_vec (ndarray): a vector of size (n_features, ), each element is the weight of corresponding feature.
        beta (scaler): the power of weights vector

    Returns:
        scaler: the weighted distance between two samples
    """
    distance_vec = np.square(s1 - s2) # Element_wise --> for each feature
    # w**beta
    weights_beta_vector = np.power(weight_vec, beta)

    weighted_distance = np.dot(distance_vec, weights_beta_vector.T)
    return weighted_distance

In [52]:
weighted_distance(Z[0],X[0], weights, 2)

18.518056841586095

In [53]:
def closest_center(sample, centers, weight_vector, beta=2):
    """it takes a sample and compare its distance to centers of clusters and return the cluster with closest center.

    Args:
        sample (ndarray): A vector that represent a data point
        centers (ndarray): A ndarray with the shape of (n_clusters, n_features) where each row represent a center of a cluster
        weight_vector (ndarray): a vector of wights for corresponding feature
        beta (scaler): a bridge that bring
    Returns:
        int: the number of cluster which is closest to the samples
    """
    d = [] # list of weighted distances
    for c in centers:
        w_d = weighted_distance(sample, c, weight_vector, beta)
        d.append(w_d)
    assigned_cluster = np.argmin(d)
    return assigned_cluster

## Solving P1

In [54]:
def u_calculation(data, centers, weights, beta = 2):
   """ Calculate U based on Z and W and our dataset
   """
   n_spl = data.shape[0] # umber of samples
   n_clu = centers.shape[0] # number of clusters
   u_matrix = np.zeros((n_spl, n_clu))
   for i, x in enumerate(data):
      l = closest_center(x, centers, weights, beta)
      u_matrix[i][l] = 1
   return u_matrix

In [55]:
U = u_calculation(X, Z, weights)
U

array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],


In [56]:
history['U'].append(U)

In [57]:
def clusters_vec(U):
    # a vector of size (n_samples, ) which each element shows the cluster that each samples is assigned to
    c_vec = np.zeros(n_samples)
    for m, u in enumerate(U):
        c_vec[m] = np.argmax(u)
    c_vec = c_vec.astype(int)
    return c_vec

In [58]:
clusters_vec(U)

array([3, 2, 2, 0, 1, 0, 0, 0, 2, 2, 3, 0, 2, 0, 1, 0, 2, 0, 0, 3, 0, 3,
       0, 1, 0, 2, 0, 3, 1, 1, 2, 2, 0, 3, 0, 0, 2, 2, 3, 2, 1, 0, 0, 0,
       0, 0, 1, 0, 3, 2, 3, 0, 0, 0, 0, 3, 1, 1, 3, 0, 0, 3, 2, 2, 3, 0,
       0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 1, 0, 3, 0, 3, 0, 0, 0, 0, 1, 0,
       0, 2, 2, 1, 0, 2, 0, 3, 3, 2, 0, 0, 2, 0, 1, 0, 0, 0, 1, 2, 0, 0,
       0, 1, 0, 0, 0, 2, 2, 2, 0, 0, 3, 0, 1, 1, 0, 2, 2, 3, 0, 0, 0, 0,
       0, 2, 1, 0, 0, 0, 0, 0, 3, 2, 3, 0, 2, 0, 2, 2, 2, 2, 0, 3, 0, 2,
       3, 0, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 3, 2, 2, 2, 0, 0, 0, 0, 2, 3,
       0, 1, 2, 2, 1, 2, 0, 3, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2,
       0, 1])

In [59]:
unique_elements, counts = np.unique(clusters_vec(U), return_counts=True)
dict(zip(unique_elements, counts))

{0: 100, 1: 24, 2: 50, 3: 26}

In [60]:
def cost_function(U, Z, W, X, beta = 2):
    """Calculate the cost function

    Args:
        U (ndarray):  U is an (M, k) matrix, ui,l is a binary variable, and ui,l = 1 indicates that record i is allocated to cluster l.
        Z (ndarray): is a set of k vectors representing the k-cluster centers of size (n_clusters, n_features)
        W (ndarray): W = [w1, w2, ..., wN ] is a set of weights of size (n_features, )
        X (ndarray): matrix of records (n_records, n_features)
        beta (int, optional): The power of elements of weights vector Defaults to 2.
    """
    P = 0 # initial value of cost

    cl_vec = clusters_vec(U)
    
    # Updating P
    for m, c in enumerate(cl_vec):
        w_d = weighted_distance(X[m], Z[c], W, beta)
        P += w_d
        P =  P.item() # to convert it to a single scaler
    return(P)


In [61]:
c_t = cost_function(U, Z, weights, X, beta = 2)
history['cost'].append(c_t)

## Solving P2

In [62]:
def clusters_dict(U):
    # Finding the index of samples in each cluster
    n_clusters = U.shape[1]
    cluster_dict = {}
    clu_vec = clusters_vec(U)

    for i in range(n_clusters):
        cluster_dict[i] = np.where(clu_vec == i)[0]
        
    return cluster_dict

In [63]:
# Z_update = np.zeros_like(Z)
# for i,ind in cluster_dict.items():
#     Z_update[i] = np.mean(X[ind], axis=0)

# Z = Z_update


In [64]:
def update_Z(U, Z, X):
    """Update Z i.e. the centers of clusters, by taking mean of teh samples in each cluster
    """
    cluster_dict = clusters_dict(U)

    new_Z = np.zeros_like(Z)
    for i,ind in cluster_dict.items():
        new_Z[i] = np.mean(X[ind], axis=0)
    
    return new_Z


In [65]:
# Update Z
Z = history['Z'][-1] # the last update of Z
weights = history['W'][-1] # the last update of weights

Z_t = update_Z(U, Z, X) # new update of Z
history['Z'].append(Z_t)

In [66]:
# Update cost
c_t = cost_function(U, Z_t, weights, X, beta = 2) 
history['cost'].append(c_t)


In [67]:
history['cost']

[497.8511571832728, 195.77152687364324]

## Solving P3

In [68]:
# # Iteration over all features to calculate Dj
# U = history['U'][-1] # the lsat update of U
# Z = history['Z'][-1] # the lsat update of Z
# cluster_dict = clusters_dict(U)

# D = []
# for j in range(n_features):
#     d_j = 0
#     for l in range(n_clusters):
#         inx_in_cluster = cluster_dict[l]
#         # Distance for feature "j" in cluster "l"
#         d_j_l =np.sum(np.square(X[inx_in_cluster][j]-Z[l][j]))
#         d_j += d_j_l

#     D.append(d_j)

# D


In [69]:
def dj(X, U, Z):
    # Iteration over all features to calculate Dj for each feature
    
    cluster_dict = clusters_dict(U)
    n_features = X.shape[1]
    n_clusters = U.shape[1]
    
    D = []
    for j in range(n_features):
        d_j = 0
        for l in range(n_clusters):
            inx_in_cluster = cluster_dict[l]
            # Distance for feature "j" in cluster "l"
            d_j_l =np.sum(np.square(X[inx_in_cluster][j]-Z[l][j]))
            d_j += d_j_l

        D.append(d_j)
    return D


In [70]:
U = history['U'][-1] # the lsat update of U
Z = history['Z'][-1] # the lsat update of Z
d = dj(X, U, Z)
d

[2176.22214506837,
 2136.2587321721894,
 1197.6594521979107,
 1127.7122319272262,
 2499.7883350746847,
 2297.9382109519115]

In [71]:
# # weights_update
# weights_update = np.zeros_like(weights)
# beta = 2

# # wherever D is zero, the corresponding weight is zero
# ind_D_zero = np.where(D == 0 )[0]
# weights_update[ind_D_zero] = 0

# ind_D_not_zero = np.where(D)[0]
# for j in ind_D_not_zero:
#     Dj_Dt = 0
    
#     for t in ind_D_not_zero:
#        Dj_Dt += (D[j] / D[t]) ** (1 / ( beta - 1) )
    
#     weights_update[j] = 1 / Dj_Dt

# weights_update

In [72]:
def weight_update(X, U, Z, weights, beta=2):

    # D calculation:
    D = dj(X, U, Z)

    
    # weights_update
    weights_upd = np.zeros_like(weights)


    # wherever D is zero, the corresponding weight is zero
    ind_D_zero = np.where(D == 0 )[0] # indexes of zero Dj
    weights_upd[ind_D_zero] = 0

    # D is not zero
    ind_D_not_zero = np.where(D)[0] ## indexes of non-zero Dj
    for j in ind_D_not_zero:
        
        Dj_Dt = 0
        for t in ind_D_not_zero:
            Dj_Dt += (D[j] / D[t]) ** (1 / ( beta - 1) )
        
        weights_upd[j] = 1 / Dj_Dt

    return weights_upd

In [73]:
# c_t = cost_function(U, Z, weights_update, X, beta = 2)
# c_t

In [74]:
U = history['U'][-1] # the lsat update of U
Z = history['Z'][-1] # the lsat update of Z
weights_t = weight_update(X, U, Z, weights, beta=2)
history['W'].append(weights_t)
c_t = cost_function(U, Z, weights_t, X, beta = 2)
history['cost'].append(c_t)

In [75]:
history['cost']

[497.8511571832728, 195.77152687364324, 172.88227678121527]

In [76]:
# do it in a loop 
# keep track of changes for weights 

## Put everything together


In [77]:
cost_difference = []

while True:
    cost_difference = np.abs(history['cost'][-1] - history['cost'][-2])
    if  cost_difference > 0.0001:

        # P1 --> update U
        Z = history['Z'][-1] # the last update of Z
        weights = history['W'][-1] # the last update of W
        U = u_calculation(X, Z, weights)
        history['U'].append(U)
        U = history['U'][-1]
        # update cost
        c_t = cost_function(U, Z, weights, X, beta = 2)
        history['cost'].append(c_t)
    else:
        break


    #P2 --> update Z
    cost_difference = np.abs(history['cost'][-1] - history['cost'][-2])
    if  cost_difference > 0.0001:
        U = history['U'][-1] # the last update of U
        Z = history['Z'][-1] # the last update of Z
        weights = history['W'][-1] # the last update of weights

        Z_t = update_Z(U, Z, X) # new update of Z
        history['Z'].append(Z_t)
        # Update cost
        c_t = cost_function(U, Z_t, weights, X, beta = 2) 
        history['cost'].append(c_t)
    else:
        break


    # P3 --> update  weights
    cost_difference = np.abs(history['cost'][-1] - history['cost'][-2])
    if  cost_difference > 0.0001:
        U = history['U'][-1] # the lsat update of U
        Z = history['Z'][-1] # the lsat update of Z
        weights_t = weight_update(X, U, Z, weights, beta=2)
        history['W'].append(weights_t)
        #update cost
        c_t = cost_function(U, Z, weights_t, X, beta = 2)
        history['cost'].append(c_t)
    else:
        break

final_weights = history['W'][-1]
print(f'Final vector of weights is:\n {final_weights} ')




Final vector of weights is:
 [0.13180226 0.13432983 0.2394493  0.25484279 0.11477661 0.12479922] 


In [78]:
history['cost']

[497.8511571832728,
 195.77152687364324,
 172.88227678121527,
 172.7946333028405,
 172.7313916622659,
 172.6104067616244,
 172.55486852243962,
 172.53210560797106,
 172.5821839502257,
 172.57912453320384,
 172.56269498879087,
 172.47716870921377,
 172.47716870921377]

In [79]:
np.sum(history['W'][-1])

1.0

In [2]:
import numpy as np

num_rows = 3
num_columns = 4

matrix = np.random.dirichlet(np.ones(num_columns), size=1)

print(matrix)
print(np.sum(matrix, axis=1))

[[2.48822922e-05 8.09846973e-02 7.04670087e-02 8.48523412e-01]]
[1.]
